### Test de l'api_brussel

In [ ]:
# api_brussels_correct.py
import requests
import json

class BrusselsAPICorrect:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        print("🔄 Génération d'un nouveau token...")
        
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        try:
            response = requests.post(token_url, data=payload)
            print(f"Token Response Status: {response.status_code}")
            
            if response.status_code == 200:
                token_data = response.json()
                new_token = token_data["access_token"]
                expires_in = token_data.get("expires_in", "Inconnu")
                print(f"✅ NOUVEAU TOKEN GÉNÉRÉ!")
                print(f"   Token: {new_token[:20]}...")
                print(f"   Expire dans: {expires_in} secondes")
                return new_token
            else:
                print(f"❌ Erreur génération token: {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ Exception token: {e}")
            return None
    
    def get_events(self, page=1, limit=10):
        """Récupère les événements avec la bonne URL"""
        if not self.access_token:
            print("❌ Aucun token disponible")
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        # URL CORRECTE comme dans la documentation
        url = f"{self.base_url}/events"
        params = {"page": page}
        
        print(f"🔍 Appel API: {url}")
        print(f"   Token: {self.access_token[:20]}...")
        
        try:
            response = requests.get(url, headers=headers, params=params)
            print(f"📡 Status Code: {response.status_code}")
            
            if response.status_code == 200:
                data = response.json()
                events = data.get("events", [])
                print(f"✅ SUCCÈS! {len(events)} événements récupérés")
                return events[:limit]  # Limite le nombre
            else:
                print(f"❌ ERREUR API: {response.text}")
                return []
                
        except Exception as e:
            print(f"❌ Exception: {e}")
            return []
    
    def search_events(self, keyword: str):
        """Recherche des événements par mot-clé"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events/search"
        params = {"keyword": keyword}
        
        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                return response.json().get("events", [])
            else:
                print(f"Erreur recherche: {response.status_code}")
                return []
        except Exception as e:
            print(f"Exception recherche: {e}")
            return []

# Test complet
if __name__ == "__main__":
    print("🧪 API BRUSSELS AGENDA - VERSION CORRIGÉE")
    print("=" * 60)
    
    api = BrusselsAPICorrect()
    
    if api.access_token:
        print("\n📅 RÉCUPÉRATION DES ÉVÉNEMENTS...")
        events = api.get_events(page=1, limit=5)
        
        if events:
            print(f"\n🎉 {len(events)} ÉVÉNEMENTS TROUVÉS:")
            print("-" * 50)
            
            for i, event in enumerate(events, 1):
                print(f"\n{i}. {event.get('title', 'Sans titre')}")
                print(f"   📍 {event.get('location', {}).get('name', 'Lieu non spécifié')}")
                print(f"   📅 {event.get('date', {}).get('start', 'Date non spécifiée')}")
                print(f"   🔗 {event.get('url', 'Pas de lien')}")
                
                # Affiche un extrait de la description
                description = event.get('description', '')
                if description:
                    print(f"   📝 {description[:100]}...")
        else:
            print("😞 Aucun événement trouvé")
            
        # Test de recherche
        print("\n🔍 TEST DE RECHERCHE 'concert'...")
        concerts = api.search_events("concert")
        print(f"🎵 {len(concerts)} concerts trouvés")
        
    else:
        print("❌ Impossible d'initialiser l'API")

In [ ]:
# api_brussels_fixed_v2.py
import requests
import json
from datetime import datetime

class BrusselsAPIWorking:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        # self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        """Génère un nouveau token d'accès"""
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, page=1, limit=10):
        """Récupère les événements avec la bonne structure"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        # url = f"{self.base_url}/events"
        url = f"{self.base_url}/events/category"
        # params = {"page": page}
        params = {"mainCategory": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            # EXTRACTION CORRECTE des événements
            events = data.get("response", {}).get("results", {}).get("event", [])
            
            print(f"📊 Statistiques:")
            print(f"   - Page: {data.get('response', {}).get('page', 1)}")
            print(f"   - Total pages: {data.get('response', {}).get('pageCount', 1)}")
            print(f"   - Total événements: {data.get('response', {}).get('resultCount', 0)}")
            print(f"   - Événements cette page: {len(events)}")
            
            return events[:limit]
        else:
            print(f"❌ Erreur API: {response.status_code}")
            return []
    
    def debug_event_structure(self, event):
        """Debug la structure d'un événement"""
        print("\n🔍 STRUCTURE DE L'ÉVÉNEMENT:")
        print(json.dumps(event, indent=2, ensure_ascii=False)[:1000] + "...")
    
    def format_event(self, event):
        """Formate un événement pour l'affichage"""
        # Debug si nécessaire
        # self.debug_event_structure(event)
        
        # Gère les traductions
        translations = event.get("translations", {})
        fr_translation = translations.get("fr", {})
        nl_translation = translations.get("nl", {})
        
        title = fr_translation.get("title") or nl_translation.get("title") or "Sans titre"
        description = fr_translation.get("shortdescr") or nl_translation.get("shortdescr") or ""
        
        # DATE - La structure est différente
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        end_date = ""
        
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", "Date non spécifiée")
                end_date = first_date.get("end", "")
        
        # Lieu
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        # Prix
        price_info = event.get("price", {})
        if isinstance(price_info, dict):
            price = price_info.get("base", "Gratuit" if event.get("is_free", False) else "Payant")
        else:
            price = "Gratuit" if event.get("is_free", False) else "Payant"
        
        return {
            "title": title,
            "description": description[:200] + "..." if len(description) > 200 else description,
            "start_date": start_date,
            "end_date": end_date,
            "location": place_name,
            "url": fr_translation.get("website") or nl_translation.get("website"),
            "price": price,
            "is_free": event.get("is_free", False)
        }
    
    def get_social_events(self, limit=6):
        """Récupère des événements sociaux spécifiques"""
        events = self.get_events(limit=limit)
        social_events = []
        
        for event in events:
            formatted = self.format_event(event)
            
            # Filtre pour événements sociaux (adaptable)
            if any(keyword in formatted['title'].lower() for keyword in 
                  ['atelier', 'workshop', 'rencontre', 'meeting', 'café', 'débat', 'discussion']):
                social_events.append(formatted)
            else:
                # Inclut tous les événements pour le POC
                social_events.append(formatted)
        
        return social_events

# Test des événements
if __name__ == "__main__":
    print("🎉 API BRUSSELS - VERSION CORRIGÉE!")
    print("=" * 60)
    
    api = BrusselsAPIWorking()
    
    if api.access_token:
        print("✅ Token valide")
        
        # Récupère les événements
        events = api.get_events(page=1, limit=6)
        
        if events:
            print(f"\n📅 AFFICHAGE DE {len(events)} ÉVÉNEMENTS:")
            print("=" * 60)
            
            for i, event in enumerate(events, 1):
                try:
                    formatted_event = api.format_event(event)
                    print(f"\n{i}. {formatted_event['title']}")
                    print(f"   📍 {formatted_event['location']}")
                    print(f"   📅 {formatted_event['start_date']}")
                    if formatted_event['end_date']:
                        print(f"   ➡️  {formatted_event['end_date']}")
                    print(f"   💰 {formatted_event['price']}")
                    if formatted_event['description']:
                        print(f"   📝 {formatted_event['description']}")
                    if formatted_event['url']:
                        print(f"   🔗 {formatted_event['url'][:50]}...")
                    
                except Exception as e:
                    print(f"\n{i}. ❌ Erreur formatage: {e}")
                    # Debug cet événement problématique
                    api.debug_event_structure(event)
                
                if i % 2 == 0:  # Séparateur tous les 2 événements
                    print("-" * 50)
                    
        else:
            print("😞 Aucun événement trouvé")
            
    else:
        print("❌ Token invalide")

### Test Premier Agent

In [ ]:

%pip uninstall -y langchain langchain-community langchain-core
%pip install langchain==0.1.16 langchain-community==0.0.38 python-dotenv requests

In [ ]:
%pip install langchain_mistralai

In [ ]:
# Version GARANTIE - Custom LLM pour éviter les problèmes d'import
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.llms.base import LLM
from typing import Optional, List, Dict, Any
import requests
import os
import json

class CustomMistralLLM(LLM):
    """Wrapper personnalisé pour l'API Mistral - Évite les problèmes d'import"""
    
    model: str = "mistral-large-latest"
    temperature: float = 0.3
    mistral_api_key: str = ""
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API Mistral: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "mistral"

print("✅ Custom Mistral LLM importé avec succès!")

In [ ]:
# social_agent_working.py
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.llms.base import LLM
from typing import Optional, List, Dict, Any
import requests
import os
import json
from dotenv import load_dotenv

load_dotenv()

# ========== CUSTOM MISTRAL LLM ==========
class CustomMistralLLM(LLM):
    model: str = "mistral-large-latest"
    temperature: float = 0.3
    mistral_api_key: str = ""
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "mistral"

# ========== BRUSSELS API ==========
class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []
    
    def format_event(self, event):
        if not event or not isinstance(event, dict):
            return {
                "title": "Événement non disponible",
                "description": "",
                "start_date": "",
                "location": "",
                "is_free": False
            }

        translations = event.get("translations", {})
        # Cherche le titre dans n'importe quelle langue
        title = "Activité sans titre"
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("title"):
                title = translations[lang]["title"]
                break
        
        # Même logique pour la description
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("shortdescr"):
                description = translations[lang]["shortdescr"]
                break
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        if dates and isinstance(dates, list) and dates:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== SOCIAL AGENT ==========
class SocialConnectionAgent:
    def __init__(self):
        # Custom LLM
        self.llm = CustomMistralLLM(mistral_api_key=os.getenv("MISTRAL_API_KEY"))
        
        # API Brussels
        self.api = BrusselsAPI()
        
        # Mémoire
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # Tools
        self.tools = self._setup_tools()
        
        # Agent LangChain
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            agent_kwargs={
                "system_message": SystemMessage(content=self._get_system_prompt()),
                "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")]
            }
        )
    
    def _get_system_prompt(self):
        return """Tu es un assistant bienveillant qui aide les personnes isolées à retrouver du lien social à Bruxelles.
        Sois encourageant et propose des activités concrètes basées sur les événements disponibles."""
    
    def _setup_tools(self):
        def find_social_activities(query: str) -> str:
            events = self.api.get_events(limit=5)
            if not events:
                return "Aucune activité sociale trouvée pour le moment."
            
            result = "🎯 ACTIVITÉS SOCIALES À BRUXELLES:\n\n"
            for i, event in enumerate(events, 1):
                formatted = self.api.format_event(event)
                result += f"{i}. {formatted['title']}\n"
                result += f"   📍 {formatted['location']}\n"
                result += f"   📅 {formatted['start_date']}\n"
                result += f"   💰 {'Gratuit' if formatted['is_free'] else 'Payant'}\n"
                if formatted['description']:
                    result += f"   📝 {formatted['description']}\n"
                result += "\n"
            
            return result
        
        return [
            Tool(
                name="RechercheActivitesSociales",
                func=find_social_activities,
                description="Recherche des activités sociales à Bruxelles pour rencontrer des gens."
            )
        ]
    
    def chat(self, message: str) -> str:
        try:
            response = self.agent.run(message)
            # Gère le cas où la réponse est un dictionnaire
            if isinstance(response, dict) and "response" in response:
                return response["response"]
            elif isinstance(response, dict) and "action_input" in response:
                return str(response)  # Fallback
            return str(response)
        except Exception as e:
            return f"Erreur: {str(e)}"

# ========== TEST ==========
print("🤖 AGENT SOCIAL CONNECTION - FONCTIONNEL!")
print("=" * 60)

agent = SocialConnectionAgent()

# Test simple
test_message = "Bonjour, je me sens seul et j'aimerais rencontrer des gens à Bruxelles"
print(f"👤: {test_message}")
response = agent.chat(test_message)
print(f"🤖: {response}")

### Travailler avec une interface pour tester plus facilement l'agent

# V1

In [ ]:
import requests
import os
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from dotenv import load_dotenv

load_dotenv()

# ========== CUSTOM MISTRAL LLM ==========
class CustomMistralLLM:
    def __init__(self):
        self.model = "mistral-large-latest"
        self.temperature = 0.3
        self.mistral_api_key = os.getenv("MISTRAL_API_KEY")
    
    def call(self, prompt: str) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"

# ========== BRUSSELS API ==========
class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events_filtered(self, filters=None, limit=6):
        """Récupère les événements avec filtres"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        # Applique les filtres si disponibles
        if filters:
            if "category" in filters:
                url = f"{self.base_url}/events/category"
                params["mainCategory"] = filters["category"]
            if "date" in filters:
                url = f"{self.base_url}/events/date" 
                params["date"] = filters["date"]
            if "keyword" in filters:
                url = f"{self.base_url}/events/search"
                params["keyword"] = filters["keyword"]
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []

    def get_categories(self):
        """Récupère les catégories disponibles"""
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events/categories"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            return data.get("results", {}).get("category", [])
        return []
    
    def get_events(self, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []
    
    def format_event(self, event):
        translations = event.get("translations", {})
        
        # Cherche le titre dans n'importe quelle langue
        title = "Activité à Bruxelles"
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("title"):
                title = translations[lang]["title"]
                break
        
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("shortdescr"):
                description = translations[lang]["shortdescr"]
                break
        
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        if dates and isinstance(dates, list) and dates:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }
    

# ========== SOCIAL AGENT SIMPLIFIÉ ==========
class GuidedSocialAgent:
    def __init__(self):
        self.llm = CustomMistralLLM()
        self.api = BrusselsAPI()
        self.conversation_stage = "welcome"  # welcome → interests → timing → results
        self.user_preferences = {
            "interests": [],
            "timing": None,  # "today", "tomorrow", "weekend", "evening", etc.
            "budget": None,  # "free", "paid", "any"
            "location": None
        }
        self.available_categories = self._load_categories()
    
    def _load_categories(self):
        """Charge les catégories disponibles - version corrigée"""
        categories = self.api.get_categories()
        category_map = {}
        
        # Debug: affiche la structure des catégories
        print(f"📊 Structure des catégories: {type(categories)}")
        if categories:
            print(f"📊 Premier élément: {categories[0] if isinstance(categories, list) else 'Not a list'}")
        
        # Si c'est une liste, traite normalement
        if isinstance(categories, list):
            for cat in categories[:10]:  # Premières 10 catégories
                if isinstance(cat, dict):
                    cat_id = cat.get("id")
                    if cat_id:
                        translations = cat.get("translations", {})
                        if isinstance(translations, dict):
                            fr_translation = translations.get("fr", {})
                            if isinstance(fr_translation, dict):
                                category_name = fr_translation.get("name", f"Catégorie {cat_id}")
                                category_map[cat_id] = category_name
        
        # Si pas de catégories ou structure différente, utilise des catégories par défaut
        if not category_map:
            category_map = {
                "1": "Musique",
                "2": "Arts", 
                "3": "Sports",
                "4": "Culture",
                "5": "Ateliers",
                "6": "Visites",
                "7": "Conférences",
                "8": "Festivals"
            }
        
        print(f"✅ Catégories chargées: {list(category_map.keys())}")
        return category_map
    
    def chat(self, message: str) -> str:
        user_message_lower = message.lower()
        
        # Met à jour les préférences basées sur le message
        self._update_preferences_from_message(user_message_lower)
        
        # Détermine l'étape de conversation
        if self.conversation_stage == "welcome":
            return self._handle_welcome_stage()
        elif self.conversation_stage == "interests":
            return self._handle_interests_stage(user_message_lower)
        elif self.conversation_stage == "timing":
            return self._handle_timing_stage(user_message_lower)
        else:
            return self._handle_results_stage()
    
    def _update_preferences_from_message(self, message: str):
        """Extrait les préférences du message utilisateur"""
        # Détection des centres d'intérêt
        interest_keywords = {
            'musique': ['musique', 'concert', 'festival', 'groupe', 'dj'],
            'art': ['art', 'exposition', 'musée', 'galerie', 'peinture'],
            'sport': ['sport', 'yoga', 'danse', 'fitness', 'randonnée'],
            'cuisine': ['cuisine', 'culinaire', 'restaurant', 'atelier cuisine'],
            'jeux': ['jeu', 'ludique', 'board game', 'escape game'],
            'nature': ['nature', 'parc', 'jardin', 'extérieur', 'balade'],
            'culture': ['culture', 'théâtre', 'cinéma', 'lecture', 'livre']
        }
        
        for interest, keywords in interest_keywords.items():
            if any(keyword in message for keyword in keywords):
                if interest not in self.user_preferences["interests"]:
                    self.user_preferences["interests"].append(interest)
        
        # Détection du timing
        timing_keywords = {
            'today': ['aujourd\'hui', 'ce soir', 'maintenant', 'tout de suite'],
            'tomorrow': ['demain', 'lendemain'],
            'weekend': ['weekend', 'samedi', 'dimanche'],
            'evening': ['soir', 'soirée', 'nuit', 'après-midi'],
            'morning': ['matin', 'matinée']
        }
        
        for timing, keywords in timing_keywords.items():
            if any(keyword in message for keyword in keywords):
                self.user_preferences["timing"] = timing
                break
        
        # Détection du budget
        if any(word in message for word in ['gratuit', 'free', 'pas cher', 'économique']):
            self.user_preferences["budget"] = "free"
        elif any(word in message for word in ['payant', 'prix', 'coût', 'budget']):
            self.user_preferences["budget"] = "paid"
    
    def _handle_welcome_stage(self):
        """Étape de bienvenue - pose la première question"""
        self.conversation_stage = "interests"
        return """
🤖 **Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles.**

Pour vous proposer les meilleures options, j'aimerais en savoir un peu plus sur vos préférences :

🎯 **Quels types d'activités vous intéressent ?**
- Musique/Concerts
- Art/Expositions  
- Sport/Activités physiques
- Cuisine/Ateliers culinaires
- Jeux/Activités ludiques
- Nature/Balades
- Culture/Théâtre

Dites-moi ce qui vous attire ! 😊
"""
    
    def _handle_interests_stage(self, message: str):
        """Gère la sélection des centres d'intérêt"""
        if not self.user_preferences["interests"]:
            return """
Je n'ai pas bien saisi vos centres d'intérêt. Pouvez-vous me dire ce qui vous plaît parmi :

🎵 **Musique** : Concerts, festivals, soirées
🎨 **Art** : Expositions, musées, galeries
⚽ **Sport** : Yoga, danse, randonnées
🍳 **Cuisine** : Ateliers, dégustations
🎮 **Jeux** : Jeux de société, escape games
🌳 **Nature** : Parcs, jardins, balades
🎭 **Culture** : Théâtre, cinéma, lectures

Qu'est-ce qui vous tente ? ✨
"""
        
        self.conversation_stage = "timing"
        interests_str = ", ".join(self.user_preferences["interests"])
        return f"""
Parfait ! Vous aimez : **{interests_str}**

🕐 **Quand souhaitez-vous faire une activité ?**
- Aujourd'hui
- Demain  
- Ce weekend
- En soirée
- En matinée

Ou dites-moi une date spécifique ! 📅
"""
    
    def _handle_timing_stage(self, message: str):
        """Gère la sélection du timing"""
        if not self.user_preferences["timing"]:
            return """
Quand envisagez-vous de participer à une activité ?

⏰ **Options :**
- **Aujourd'hui** : Activités immédiates
- **Demain** : Planification courte
- **Weekend** : Samedi ou dimanche  
- **Soirée** : Après 18h
- **Matinée** : Avant midi

Quel moment vous convient ? 😊
"""
        
        # Passe à la recherche des résultats
        self.conversation_stage = "results"
        return self._handle_results_stage()
    
    def _handle_results_stage(self):
        """Effectue la recherche et présente les résultats"""
        # Construit les filtres pour l'API
        filters = {}
        
        # Filtre par catégorie basé sur les intérêts
        interest_to_category = {
            'musique': 1,   # Musique
            'art': 2,       # Arts
            'sport': 3,     # Sports
            'culture': 4,   # Culture générale
        }
        
        for interest in self.user_preferences["interests"]:
            if interest in interest_to_category:
                filters["category"] = interest_to_category[interest]
                break
        
        # Filtre par mot-clé pour les autres intérêts
        if not filters.get("category") and self.user_preferences["interests"]:
            primary_interest = self.user_preferences["interests"][0]
            filters["keyword"] = primary_interest
        
        # Récupère les événements filtrés
        events = self.api.get_events_filtered(filters, limit=8)
        
        if not events:
            return self._handle_no_results()
        
        # Formate les résultats
        return self._format_final_results(events)
    
    def _handle_no_results(self):
        """Gère le cas où aucun événement n'est trouvé"""
        alternative_filters = {}
        events = self.api.get_events_filtered(alternative_filters, limit=5)
        
        if not events:
            return """
Désolé, je n'ai trouvé aucun événement correspondant à vos critères. 

🔄 **Suggestions :**
- Élargissez vos centres d'intérêt
- Essayez une autre période
- Consultez les sites comme Meetup.com ou Facebook Events

Voulez-vous recommencer la recherche ? 🔍
"""
        
        return self._format_final_results(events, is_fallback=True)
    
    def _format_final_results(self, events, is_fallback=False):
        """Formate les résultats finaux"""
        if is_fallback:
            result = "🔍 **Je n'ai pas trouvé d'événements exacts, mais voici des suggestions :**\n\n"
        else:
            interests_str = ", ".join(self.user_preferences["interests"])
            timing_str = self.user_preferences["timing"] or "prochainement"
            result = f"🎯 **Activités pour vous ({interests_str}) - {timing_str}:**\n\n"
        
        for i, event in enumerate(events, 1):
            formatted = self.api.format_event(event)
            result += f"**{i}. {formatted['title']}**\n"
            result += f"   📍 {formatted['location']}\n"
            result += f"   📅 {formatted['start_date']}\n"
            result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
            if formatted['description']:
                result += f"   📝 {formatted['description']}\n"
            
            # Conseil social personnalisé
            social_tip = self._get_social_tip(formatted['title'])
            if social_tip:
                result += f"   💡 {social_tip}\n"
            
            result += "\n"
        
        result += """
🔄 **Pour une nouvelle recherche :**
Tapez "recommencer" ou dites-moi d'autres préférences !

J'espère que ces activités vous aideront à rencontrer de nouvelles personnes ! 🤝
"""
        
        return result
    
    def _get_social_tip(self, title):
        """Retourne un conseil social basé sur le type d'activité"""
        title_lower = title.lower()
        
        if any(word in title_lower for word in ['visite', 'guidée', 'tour']):
            return "Idéal pour discuter avec d'autres participants pendant la visite !"
        elif any(word in title_lower for word in ['atelier', 'workshop']):
            return "Parfait pour collaborer et échanger avec les autres !"
        elif any(word in title_lower for word in ['cours', 'formation']):
            return "Excellent pour rencontrer des personnes partageant les mêmes intérêts !"
        elif any(word in title_lower for word in ['concert', 'festival']):
            return "Ambiance conviviale pour discuter autour de la musique !"
        
        return "Bonne occasion de rencontrer des gens partageant vos centres d'intérêt !"
    
    def reset_conversation(self):
        """Réinitialise la conversation"""
        self.conversation_stage = "welcome"
        self.user_preferences = {
            "interests": [],
            "timing": None,
            "budget": None, 
            "location": None
        }
print("✅ Agent social chargé avec succès!")

# V2 

In [ ]:
import requests
import os
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from dotenv import load_dotenv

load_dotenv()

# ========== CUSTOM MISTRAL LLM ==========
class CustomMistralLLM:
    def __init__(self):
        self.model = "mistral-large-latest"
        self.temperature = 0.3
        self.mistral_api_key = os.getenv("MISTRAL_API_KEY")
    
    def call(self, prompt: str) -> str:
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.mistral_api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                return f"Erreur API: {response.status_code}"
        except Exception as e:
            return f"Exception: {str(e)}"

# ========== BRUSSELS API ==========
class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events_filtered(self, filters=None, limit=6):
        """Récupère les événements avec filtres"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        # Applique les filtres si disponibles
        if filters:
            if "category" in filters:
                url = f"{self.base_url}/events/category"
                params["mainCategory"] = filters["category"]
            if "date" in filters:
                url = f"{self.base_url}/events/date" 
                params["date"] = filters["date"]
            if "keyword" in filters:
                url = f"{self.base_url}/events/search"
                params["keyword"] = filters["keyword"]
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []

    def get_categories(self):
        """Récupère les catégories disponibles"""
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events/categories"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            return data.get("results", {}).get("category", [])
        return []
    
    def get_events(self, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            events = data.get("response", {}).get("results", {}).get("event", [])
            return events[:limit]
        return []
    
    def format_event(self, event):
        translations = event.get("translations", {})
        
        # Cherche le titre dans n'importe quelle langue
        title = "Activité à Bruxelles"
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("title"):
                title = translations[lang]["title"]
                break
        
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and translations[lang].get("shortdescr"):
                description = translations[lang]["shortdescr"]
                break
        
        dates = event.get("dates", [])
        start_date = "Date non spécifiée"
        if dates and isinstance(dates, list) and dates:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        place = event.get("place", {})
        place_name = place.get("name", "Lieu non spécifié") if isinstance(place, dict) else str(place)
        
        return {
            "title": title,
            "description": description[:150] + "..." if len(description) > 150 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== SOCIAL AGENT ==========
class GuidedSocialAgent:
    def __init__(self):
        self.llm = CustomMistralLLM()
        self.api = BrusselsAPI()
        self.conversation_stage = "welcome"  # welcome → interests → timing → results
        self.user_preferences = {
            "interests": [],
            "timing": None,  # "today", "tomorrow", "weekend", "evening", etc.
            "budget": None,  # "free", "paid", "any"
            "location": None
        }
        self.available_categories = self._load_categories()
    
    def _load_categories(self):
        """Charge les catégories disponibles - version corrigée"""
        categories = self.api.get_categories()
        category_map = {}
        
        # Si c'est une liste, traite normalement
        if isinstance(categories, list):
            for cat in categories[:10]:  # Premières 10 catégories
                if isinstance(cat, dict):
                    cat_id = cat.get("id")
                    if cat_id:
                        translations = cat.get("translations", {})
                        if isinstance(translations, dict):
                            fr_translation = translations.get("fr", {})
                            if isinstance(fr_translation, dict):
                                category_name = fr_translation.get("name", f"Catégorie {cat_id}")
                                category_map[cat_id] = category_name
        
        # Si pas de catégories ou structure différente, utilise des catégories par défaut
        if not category_map:
            category_map = {
                "1": "Musique",
                "2": "Arts", 
                "3": "Sports",
                "4": "Culture",
                "5": "Ateliers",
                "6": "Visites",
                "7": "Conférences",
                "8": "Festivals"
            }
        
        return category_map
    
    def chat(self, message: str) -> str:
        user_message_lower = message.lower()
        
        # Met à jour les préférences basées sur le message
        self._update_preferences_from_message(user_message_lower)
        
        # Détermine l'étape de conversation
        if self.conversation_stage == "welcome":
            return self._handle_welcome_stage()
        elif self.conversation_stage == "interests":
            return self._handle_interests_stage(user_message_lower)
        elif self.conversation_stage == "timing":
            return self._handle_timing_stage(user_message_lower)
        else:
            return self._handle_results_stage()
    
    def _update_preferences_from_message(self, message: str):
        """Extrait les préférences du message utilisateur"""
        # Détection des centres d'intérêt
        interest_keywords = {
            'musique': ['musique', 'concert', 'festival', 'groupe', 'dj'],
            'art': ['art', 'exposition', 'musée', 'galerie', 'peinture'],
            'sport': ['sport', 'yoga', 'danse', 'fitness', 'randonnée'],
            'cuisine': ['cuisine', 'culinaire', 'restaurant', 'atelier cuisine'],
            'jeux': ['jeu', 'ludique', 'board game', 'escape game'],
            'nature': ['nature', 'parc', 'jardin', 'extérieur', 'balade'],
            'culture': ['culture', 'théâtre', 'cinéma', 'lecture', 'livre']
        }
        
        for interest, keywords in interest_keywords.items():
            if any(keyword in message for keyword in keywords):
                if interest not in self.user_preferences["interests"]:
                    self.user_preferences["interests"].append(interest)
        
        # Détection du timing
        timing_keywords = {
            'today': ["aujourd'hui", 'ce soir', 'maintenant', 'tout de suite'],
            'tomorrow': ['demain', 'lendemain'],
            'weekend': ['weekend', 'samedi', 'dimanche'],
            'evening': ['soir', 'soirée', 'nuit', 'après-midi'],
            'morning': ['matin', 'matinée']
        }
        
        for timing, keywords in timing_keywords.items():
            if any(keyword in message for keyword in keywords):
                self.user_preferences["timing"] = timing
                break
        
        # Détection du budget
        if any(word in message for word in ['gratuit', 'free', 'pas cher', 'économique']):
            self.user_preferences["budget"] = "free"
        elif any(word in message for word in ['payant', 'prix', 'coût', 'budget']):
            self.user_preferences["budget"] = "paid"
    
    def _handle_welcome_stage(self):
        """Étape de bienvenue - pose la première question"""
        self.conversation_stage = "interests"
        return """
🤖 **Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles.**

Pour vous proposer les meilleures options, j'aimerais en savoir un peu plus sur vos préférences :

🎯 **Quels types d'activités vous intéressent ?**
- Musique/Concerts
- Art/Expositions  
- Sport/Activités physiques
- Cuisine/Ateliers culinaires
- Jeux/Activités ludiques
- Nature/Balades
- Culture/Théâtre

Dites-moi ce qui vous attire ! 😊
"""
    
    def _handle_interests_stage(self, message: str):
        """Gère la sélection des centres d'intérêt"""
        if not self.user_preferences["interests"]:
            return """
Je n'ai pas bien saisi vos centres d'intérêt. Pouvez-vous me dire ce qui vous plaît parmi :

🎵 **Musique** : Concerts, festivals, soirées
🎨 **Art** : Expositions, musées, galeries
⚽ **Sport** : Yoga, danse, randonnées
🍳 **Cuisine** : Ateliers, dégustations
🎮 **Jeux** : Jeux de société, escape games
🌳 **Nature** : Parcs, jardins, balades
🎭 **Culture** : Théâtre, cinéma, lectures

Qu'est-ce qui vous tente ? ✨
"""
        
        self.conversation_stage = "timing"
        interests_str = ", ".join(self.user_preferences["interests"])
        return f"""
Parfait ! Vous aimez : **{interests_str}**

🕐 **Quand souhaitez-vous faire une activité ?**
- Aujourd'hui
- Demain  
- Ce weekend
- En soirée
- En matinée

Ou dites-moi une date spécifique ! 📅
"""
    
    def _handle_timing_stage(self, message: str):
        """Gère la sélection du timing"""
        if not self.user_preferences["timing"]:
            return """
Quand envisagez-vous de participer à une activité ?

⏰ **Options :**
- **Aujourd'hui** : Activités immédiates
- **Demain** : Planification courte
- **Weekend** : Samedi ou dimanche  
- **Soirée** : Après 18h
- **Matinée** : Avant midi

Quel moment vous convient ? 😊
"""
        
        # Passe à la recherche des résultats
        self.conversation_stage = "results"
        return self._handle_results_stage()
    
    def _handle_results_stage(self):
        """Effectue la recherche et présente les résultats"""
        # Construit les filtres pour l'API
        filters = {}
        
        # Filtre par catégorie basé sur les intérêts
        interest_to_category = {
            'musique': 1,   # Musique
            'art': 2,       # Arts
            'sport': 3,     # Sports
            'culture': 4,   # Culture générale
        }
        
        for interest in self.user_preferences["interests"]:
            if interest in interest_to_category:
                filters["category"] = interest_to_category[interest]
                break
        
        # Filtre par mot-clé pour les autres intérêts
        if not filters.get("category") and self.user_preferences["interests"]:
            primary_interest = self.user_preferences["interests"][0]
            filters["keyword"] = primary_interest
        
        # Récupère les événements filtrés
        events = self.api.get_events_filtered(filters, limit=8)
        
        if not events:
            return self._handle_no_results()
        
        # Formate les résultats
        return self._format_final_results(events)
    
    def _handle_no_results(self):
        """Gère le cas où aucun événement n'est trouvé"""
        alternative_filters = {}
        events = self.api.get_events_filtered(alternative_filters, limit=5)
        
        if not events:
            return """
Désolé, je n'ai trouvé aucun événement correspondant à vos critères. 

🔄 **Suggestions :**
- Élargissez vos centres d'intérêt
- Essayez une autre période
- Consultez les sites comme Meetup.com ou Facebook Events

Voulez-vous recommencer la recherche ? 🔍
"""
        
        return self._format_final_results(events, is_fallback=True)
    
    def _format_final_results(self, events, is_fallback=False):
        """Formate les résultats finaux"""
        if is_fallback:
            result = "🔍 **Je n'ai pas trouvé d'événements exacts, mais voici des suggestions :**\n\n"
        else:
            interests_str = ", ".join(self.user_preferences["interests"])
            timing_str = self.user_preferences["timing"] or "prochainement"
            result = f"🎯 **Activités pour vous ({interests_str}) - {timing_str}:**\n\n"
        
        for i, event in enumerate(events, 1):
            formatted = self.api.format_event(event)
            result += f"**{i}. {formatted['title']}**\n"
            result += f"   📍 {formatted['location']}\n"
            result += f"   📅 {formatted['start_date']}\n"
            result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
            if formatted['description']:
                result += f"   📝 {formatted['description']}\n"
            
            # Conseil social personnalisé
            social_tip = self._get_social_tip(formatted['title'])
            if social_tip:
                result += f"   💡 {social_tip}\n"
            
            result += "\n"
        
        result += """
🔄 **Pour une nouvelle recherche :**
Tapez "recommencer" ou dites-moi d'autres préférences !

J'espère que ces activités vous aideront à rencontrer de nouvelles personnes ! 🤝
"""
        
        return result
    
    def _get_social_tip(self, title):
        """Retourne un conseil social basé sur le type d'activité"""
        title_lower = title.lower()
        
        if any(word in title_lower for word in ['visite', 'guidée', 'tour']):
            return "Idéal pour discuter avec d'autres participants pendant la visite !"
        elif any(word in title_lower for word in ['atelier', 'workshop']):
            return "Parfait pour collaborer et échanger avec les autres !"
        elif any(word in title_lower for word in ['cours', 'formation']):
            return "Excellent pour rencontrer des personnes partageant les mêmes intérêts !"
        elif any(word in title_lower for word in ['concert', 'festival']):
            return "Ambiance conviviale pour discuter autour de la musique !"
        
        return "Bonne occasion de rencontrer des gens partageant vos centres d'intérêt !"
    
    def reset_conversation(self):
        """Réinitialise la conversation"""
        self.conversation_stage = "welcome"
        self.user_preferences = {
            "interests": [],
            "timing": None,
            "budget": None, 
            "location": None
        }
print("✅ Agent social chargé avec succès!")

✅ Agent social chargé avec succès!


In [21]:
# ========== BRUSSELS API AMÉLIORÉE ==========
class BrusselsAPI:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, limit=6):
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        url = f"{self.base_url}/events"
        params = {"page": 1}
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            print(f"DEBUG - Structure API: {list(data.keys())}")  # Debug
            events = data.get("response", {}).get("results", {}).get("event", [])
            print(f"DEBUG - {len(events)} événements bruts trouvés")  # Debug
            return events[:limit]
        else:
            print(f"DEBUG - Erreur API: {response.status_code}")
            return []
    
    def format_event(self, event):
        """Version améliorée de l'extraction des données"""
        print(f"DEBUG - Structure event: {list(event.keys())}")  # Debug
        
        # Essai d'extraction du titre
        title = "Activité intéressante"
        
        # Méthode 1: Translations
        translations = event.get("translations", {})
        if translations:
            for lang in ["fr", "nl", "en", "de"]:
                if lang in translations:
                    lang_data = translations[lang]
                    if isinstance(lang_data, dict) and lang_data.get("title"):
                        title = lang_data["title"]
                        break
        
        # Méthode 2: Title direct
        if title == "Activité intéressante" and event.get("title"):
            title = event["title"]
        
        # Description
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations:
                lang_data = translations[lang]
                if isinstance(lang_data, dict) and lang_data.get("shortdescr"):
                    description = lang_data["shortdescr"]
                    break
        
        # Date améliorée
        dates = event.get("dates", [])
        start_date = "Date à confirmer"
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
                # Nettoyer la date si c'est un timestamp
                if "T" in str(start_date):
                    start_date = str(start_date).split("T")[0]
        
        # Lieu amélioré
        place = event.get("place", {})
        place_name = "Bruxelles"
        if isinstance(place, dict):
            place_name = place.get("name", "Bruxelles")
            # Essayer d'autres champs pour le lieu
            if place_name == "Bruxelles" and place.get("translations"):
                place_translations = place.get("translations", {})
                for lang in ["fr", "nl", "en"]:
                    if lang in place_translations:
                        place_name = place_translations[lang].get("name", "Bruxelles")
                        if place_name != "Bruxelles":
                            break
        
        return {
            "title": title[:80] + "..." if len(title) > 80 else title,
            "description": description[:120] + "..." if len(description) > 120 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== AGENT AMÉLIORÉ ==========
class ImprovedSocialAgent:
    def __init__(self):
        self.api = BrusselsAPI()
    
    def chat(self, message: str) -> str:
        try:
            # Analyse le message pour personnaliser la réponse
            user_msg = message.lower()
            
            if any(word in user_msg for word in ['ennui', 'ennuie', 'solitude', 'seul', 'isolé']):
                intro = "😊 Je vois que vous cherchez à sortir de l'ennui ! Voici des activités sociales :\n\n"
            elif any(word in user_msg for word in ['musique', 'concert', 'festival']):
                intro = "🎵 Des activités musicales pour vous :\n\n"
            elif any(word in user_msg for word in ['art', 'culture', 'exposition', 'musée']):
                intro = "🎨 Des activités artistiques et culturelles :\n\n"
            else:
                intro = "🎯 Voici des activités intéressantes à Bruxelles :\n\n"
            
            events = self.api.get_events(limit=6)
            
            if not events:
                return "Aucun événement trouvé pour le moment. Essayez de regarder sur https://agenda.brussels"
            
            result = intro
            
            for i, event in enumerate(events, 1):
                formatted = self.api.format_event(event)
                result += f"**{i}. {formatted['title']}**\n"
                result += f"   📍 {formatted['location']}\n"
                result += f"   📅 {formatted['start_date']}\n"
                result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
                if formatted['description']:
                    result += f"   📝 {formatted['description']}\n"
                result += "\n"
            
            result += "💡 **Conseil social :** N'hésitez pas à discuter avec les autres participants !\n"
            result += "🔄 **Nouvelle recherche :** Dites-moi ce qui vous intéresse !"
            
            return result
            
        except Exception as e:
            return f"Je rencontre un petit problème technique. Mais vous pouvez consulter directement : https://agenda.brussels"

# Dans create_chat_interface(), remplacez :
# agent = SimpleSocialAgent() 
# par :
# agent = ImprovedSocialAgent()

In [24]:
# ========== BRUSSELS API AVEC DEBUG ==========
class BrusselsAPIDebug:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events_with_keyword(self, keyword=None, limit=6):
        """Récupère les événements avec recherche par mot-clé"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        if keyword:
            # Recherche par mot-clé
            url = f"{self.base_url}/events/search"
            params = {"keyword": keyword, "page": 1}
        else:
            # Tous les événements
            url = f"{self.base_url}/events"
            params = {"page": 1}
        
        print(f"DEBUG - Appel API: {url} avec params: {params}")
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            print(f"DEBUG - Structure réponse: {list(data.keys())}")
            
            # Essayer différentes structures possibles
            events = []
            if "response" in data:
                response_data = data["response"]
                if "results" in response_data:
                    results = response_data["results"]
                    if "event" in results:
                        events = results["event"]
                    else:
                        events = results.get("events", [])
                else:
                    events = response_data.get("events", [])
            else:
                events = data.get("events", [])
            
            print(f"DEBUG - {len(events)} événements trouvés")
            
            # Debug du premier événement
            if events and len(events) > 0:
                print(f"DEBUG - Premier événement: {list(events[0].keys())}")
                if 'translations' in events[0]:
                    print(f"DEBUG - Translations: {events[0]['translations']}")
            
            return events[:limit]
        else:
            print(f"DEBUG - Erreur API: {response.status_code} - {response.text}")
            return []
    
    def format_event_debug(self, event):
        """Extraction améliorée avec debug"""
        print(f"DEBUG - Formatage événement: {event.get('id', 'no-id')}")
        
        # Essayer plusieurs méthodes pour le titre
        title = "Activité"
        
        # Méthode 1: Translations directes
        translations = event.get("translations", {})
        print(f"DEBUG - Translations disponibles: {list(translations.keys())}")
        
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations:
                lang_data = translations[lang]
                print(f"DEBUG - Lang {lang}: {lang_data}")
                if isinstance(lang_data, dict):
                    if lang_data.get("title"):
                        title = lang_data["title"]
                        print(f"DEBUG - Titre trouvé ({lang}): {title}")
                        break
                    elif lang_data.get("name"):
                        title = lang_data["name"]
                        print(f"DEBUG - Nom trouvé ({lang}): {title}")
                        break
        
        # Méthode 2: Champs directs
        if title == "Activité":
            for field in ["title", "name", "label"]:
                if event.get(field):
                    title = event[field]
                    print(f"DEBUG - Titre direct ({field}): {title}")
                    break
        
        # Description
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and isinstance(translations[lang], dict):
                desc = translations[lang].get("shortdescr") or translations[lang].get("description")
                if desc:
                    description = desc
                    break
        
        # Date
        dates = event.get("dates", [])
        start_date = "Date à confirmer"
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
        
        # Lieu
        place = event.get("place", {})
        place_name = "Bruxelles"
        if isinstance(place, dict):
            place_name = place.get("name", "Bruxelles")
            if place_name == "Bruxelles" and "translations" in place:
                for lang in ["fr", "nl", "en"]:
                    if lang in place["translations"]:
                        place_name = place["translations"][lang].get("name", "Bruxelles")
                        if place_name != "Bruxelles":
                            break
        
        return {
            "title": title[:60] + "..." if len(title) > 60 else title,
            "description": description[:100] + "..." if len(description) > 100 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== AGENT INTELLIGENT ==========
class SmartSocialAgent:
    def __init__(self):
        self.api = BrusselsAPIDebug()
    
    def chat(self, message: str) -> str:
        try:
            user_msg = message.lower()
            
            # Détection des intentions
            if any(word in user_msg for word in ['film', 'cinéma', 'movie', 'projection']):
                keyword = "cinema"
                intro = "🎬 **Voici des activités cinéma et films :**\n\n"
            elif any(word in user_msg for word in ['musique', 'concert', 'festival', 'groupe']):
                keyword = "concert"
                intro = "🎵 **Activités musicales :**\n\n"
            elif any(word in user_msg for word in ['art', 'exposition', 'musée', 'galerie']):
                keyword = "art"
                intro = "🎨 **Activités artistiques :**\n\n"
            elif any(word in user_msg for word in ['ennui', 'seul', 'solitude', 'rencontrer']):
                keyword = None  # Tous les événements
                intro = "😊 **Des activités pour socialiser :**\n\n"
            else:
                keyword = None
                intro = "🎯 **Activités à Bruxelles :**\n\n"
            
            # Recherche avec ou sans mot-clé
            events = self.api.get_events_with_keyword(keyword, limit=6)
            
            if not events:
                if keyword:
                    # Si recherche spécifique échoue, essaie tous les événements
                    events = self.api.get_events_with_keyword(None, limit=6)
                    intro = f"🔍 Aucun résultat pour '{keyword}', voici d'autres activités :\n\n"
                else:
                    return "❌ Aucun événement trouvé. Essayez https://agenda.brussels"
            
            result = intro
            
            for i, event in enumerate(events, 1):
                formatted = self.api.format_event_debug(event)
                result += f"**{i}. {formatted['title']}**\n"
                result += f"   📍 {formatted['location']}\n"
                result += f"   📅 {formatted['start_date']}\n"
                result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
                if formatted['description']:
                    result += f"   📝 {formatted['description']}\n"
                result += "\n"
            
            result += "💬 **Conseil :** Ces activités sont d'excellentes occasions de rencontrer des gens !\n"
            result += "🔄 **Nouvelle recherche :** Dites-moi ce qui vous plaît !"
            
            return result
            
        except Exception as e:
            return f"⚠️ Problème technique. Consultez directement : https://agenda.brussels\nErreur: {str(e)}"

# Dans create_chat_interface(), remplacez par :
# agent = SmartSocialAgent()

# On reprend avec un agent simple pcq sinon il donne pas de résultat 

### et de cette maniére on essaie de préciser les infos des API qu'on a 

In [20]:
# ========== SOCIAL AGENT ULTRA SIMPLE ==========
class SimpleSocialAgent:
    def __init__(self):
        self.api = BrusselsAPI()
    
    def chat(self, message: str) -> str:
        try:
            # Récupère simplement les derniers événements
            events = self.api.get_events(limit=6)
            
            if not events:
                return "Aucun événement trouvé pour le moment. Essayez plus tard !"
            
            # Formatage simple
            result = "🎯 **Voici des activités à Bruxelles :**\n\n"
            
            for i, event in enumerate(events, 1):
                formatted = self.api.format_event(event)
                result += f"**{i}. {formatted['title']}**\n"
                result += f"   📍 {formatted['location']}\n"
                result += f"   📅 {formatted['start_date']}\n"
                result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
                if formatted['description']:
                    result += f"   📝 {formatted['description']}\n"
                result += "\n"
            
            result += "💡 **Conseil :** Ces activités sont parfaites pour rencontrer du monde !"
            return result
            
        except Exception as e:
            return f"Je rencontre un problème technique. Erreur: {str(e)}"

# V1

In [ ]:
# Initialisation de l'agent
agent = GuidedSocialAgent()

# Création de l'interface
chat_output = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'overflow_y': 'scroll', 'padding': '10px'})
message_input = widgets.Text(placeholder='Tapez votre message ici...', layout={'width': '70%'})
send_button = widgets.Button(description='Envoyer ✨', button_style='success', layout={'width': '20%'})
clear_button = widgets.Button(description='Effacer', button_style='warning', layout={'width': '10%'})

# Style CSS
css_style = """
<style>
.chat-message { 
    margin: 10px 0; 
    padding: 12px; 
    border-radius: 15px; 
    max-width: 80%; 
    word-wrap: break-word;
}
.user-message { 
    background: #e3f2fd; 
    margin-left: 20%; 
    border-bottom-right-radius: 5px;
}
.bot-message { 
    background: #f5f5f5; 
    margin-right: 20%; 
    border-bottom-left-radius: 5px;
}
.message-sender { 
    font-weight: bold; 
    margin-bottom: 5px; 
    color: #333;
}
</style>
"""

# Affiche le style CSS
display(HTML(css_style))

# Fonctions
def send_message(_):
    message = message_input.value.strip()
    if not message:
        return
    
    with chat_output:
        # Affiche le message utilisateur
        display(HTML(f"""
        <div class="chat-message user-message">
            <div class="message-sender">👤 Vous</div>
            {message}
        </div>
        """))
    
    # Désactive le bouton pendant le traitement
    send_button.disabled = True
    send_button.description = "Réflexion..."
    
    # Réponse de l'agent
    try:
        if message.lower() in ['recommencer', 'reset', 'nouvelle recherche']:
            agent.reset_conversation()
            with chat_output:
                display(HTML("""
                <div class="chat-message bot-message">
                    <div class="message-sender">🔄 Reset</div>
                    Conversation réinitialisée ! Prêt pour une nouvelle recherche 🔍
                </div>
                """))
        else :
            response = agent.chat(message)
        with chat_output:
            # Affiche la réponse de l'agent
            display(HTML(f"""
            <div class="chat-message bot-message">
                <div class="message-sender">🤖 Agent Social</div>
                {response.replace(chr(10), '<br>')}
            </div>
            """))
    except Exception as e:
        with chat_output:
            display(HTML(f"""
            <div class="chat-message bot-message">
                <div class="message-sender">❌ Erreur</div>
                Désolé, une erreur s'est produite: {str(e)}
            </div>
            """))
        print(f"DEBUG Erreur: {e}")  # Pour debug
    
    # Réactive le bouton
    send_button.disabled = False
    send_button.description = "Envoyer ✨"
    message_input.value = ''
    
    # Scroll vers le bas
    chat_output._scroll_to_bottom()

def clear_chat(_):
    chat_output.clear_output()
    agent.conversation_history = []
    with chat_output:
        display(HTML("""
        <div class="chat-message bot-message">
            <div class="message-sender">🤖 Agent Social</div>
            Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. 
            Dites-moi ce qui vous intéresse ou comment vous vous sentez aujourd'hui.
        </div>
        """))

def on_enter(_):
    send_message(None)

# Assignation des événements
send_button.on_click(send_message)
clear_button.on_click(clear_chat)
message_input.on_submit(on_enter)

# Affichage de l'interface
display(HTML("<h2>🤖 Social Connection Agent</h2>"))
display(HTML("<p><em>Je vous aide à trouver des activités sociales à Bruxelles 🌟</em></p>"))
display(chat_output)
display(widgets.HBox([message_input, send_button, clear_button]))

# Message de bienvenue initial
with chat_output:
    display(HTML("""
    <div class="chat-message bot-message">
        <div class="message-sender">🤖 Agent Social</div>
        Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. <br><br>
        💡 <strong>Exemples de questions:</strong><br>
        • "Je me sens seul, que faire ?"<br>
        • "Je cherche des activités gratuites"<br>
        • "J'aime la musique, que me conseilles-tu ?"<br>
        • "Quels événements y a-t-il ce weekend ?"
    </div>
    """))

# V2 interface

In [25]:
"""Crée l'interface de chat corrigée"""
def create_chat_interface():

    # Initialisation de l'agent
    # agent = GuidedSocialAgent()
    # agent = SimpleSocialAgent()
    # agent = ImprovedSocialAgent()
    agent = SmartSocialAgent()

    # Création des widgets
    chat_output = widgets.Output(layout={
        'border': '1px solid black', 
        'height': '400px', 
        'overflow_y': 'auto',
        'padding': '10px'
    })

    message_input = widgets.Text(
        placeholder='Tapez votre message ici...', 
        layout={'width': '70%'}
    )

    send_button = widgets.Button(
        description='Envoyer ✨', 
        button_style='success', 
        layout={'width': '20%'}
    )

    clear_button = widgets.Button(
        description='Effacer', 
        button_style='warning', 
        layout={'width': '10%'}
    )

    # Style CSS
    css_style = """
    <style>
    .chat-message { 
        margin: 10px 0; 
        padding: 12px; 
        border-radius: 15px; 
        max-width: 80%; 
        word-wrap: break-word;
    }
    .user-message { 
        background: #e3f2fd; 
        margin-left: 20%; 
        border-bottom-right-radius: 5px;
    }
    .bot-message { 
        background: #f5f5f5; 
        margin-right: 20%; 
        border-bottom-left-radius: 5px;
    }
    .message-sender { 
        font-weight: bold; 
        margin-bottom: 5px; 
        color: #333;
    }
    </style>
    """

    # Fonction pour envoyer un message
    def send_message(_):
        message = message_input.value.strip()
        if not message:
            return
        
        # Affiche le message utilisateur
        with chat_output:
            display(HTML(f"""
            <div class="chat-message user-message">
                <div class="message-sender">👤 Vous</div>
                {message}
            </div>
            """))
        
        # Désactive le bouton pendant le traitement
        send_button.disabled = True
        send_button.description = "Réflexion..."
        
        try:
            # Gère les commandes spéciales
            if message.lower() in ['recommencer', 'reset', 'nouvelle recherche', 'start']:
                agent.reset_conversation()
                response = "Conversation réinitialisée ! Prêt pour une nouvelle recherche 🔍"
            else:
                # Réponse normale de l'agent
                response = agent.chat(message)
            
            # Affiche la réponse
            with chat_output:
                display(HTML(f"""
                <div class="chat-message bot-message">
                    <div class="message-sender">🤖 Agent Social</div>
                    {response.replace(chr(10), '<br>')}
                </div>
                """))
                
        except Exception as e:
            with chat_output:
                display(HTML(f"""
                <div class="chat-message bot-message">
                    <div class="message-sender">❌ Erreur</div>
                    Désolé, une erreur s'est produite. Réessayez !
                </div>
                """))
            print(f"DEBUG Erreur: {e}")
        
        # Réactive le bouton
        send_button.disabled = False
        send_button.description = "Envoyer ✨"
        message_input.value = ''

    # Fonction pour effacer le chat
    def clear_chat(_):
        chat_output.clear_output()
        agent.reset_conversation()
        with chat_output:
            display(HTML("""
            <div class="chat-message bot-message">
                <div class="message-sender">🤖 Agent Social</div>
                Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. 
                Dites-moi ce qui vous intéresse ou comment vous vous sentez aujourd'hui.
            </div>
            """))

    # Fonction pour la touche Entrée
    def on_enter(_):
        send_message(None)

    # Assignation des événements
    send_button.on_click(send_message)
    clear_button.on_click(clear_chat)
    message_input.on_submit(on_enter)

    # Affichage de l'interface
    display(HTML(css_style))
    display(HTML("<h2>🤖 Social Connection Agent</h2>"))
    display(HTML("<p><em>Je vous aide à trouver des activités sociales à Bruxelles 🌟</em></p>"))
    display(chat_output)
    display(widgets.HBox([message_input, send_button, clear_button]))
    
    # Message de bienvenue initial
    with chat_output:
        display(HTML("""
        <div class="chat-message bot-message">
            <div class="message-sender">🤖 Agent Social</div>
            Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. <br><br>
            💡 <strong>Exemples de questions:</strong><br>
            • "Je me sens seul, que faire ?"<br>
            • "Je cherche des activités gratuites"<br>
            • "J'aime la musique, que me conseilles-tu ?"<br>
            • "Quels événements y a-t-il ce weekend ?"<br><br>
            Tapez votre message ci-dessous pour commencer ! 🚀
        </div>
        """))

# Lancement de l'interface
print("✅ Interface sociale prête !")
create_chat_interface()


✅ Interface sociale prête !


C:\Users\ziani\AppData\Local\Temp\ipykernel_21636\3830999771.py:135: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  message_input.on_submit(on_enter)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

#### Smart Social Agent est bon mais pas assez, il faut un agent plus varié, plus contextuels, ...
------
##### WorkingSocialAgent

In [31]:
# ========== BRUSSELS API CORRECTE ==========
class BrusselsAPICorrected:
    def __init__(self):
        self.consumer_key = "SQ6_SSmEc625N1kunLbWsUjVrJIa"
        self.consumer_secret = "uTVflA9AQfhZJnvE3_safE1_0mka"
        self.base_url = "https://api.brussels:443/api/agenda/0.0.1"
        self.access_token = self.get_new_token()
    
    def get_new_token(self):
        token_url = "https://api.brussels:443/api/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": self.consumer_key,
            "client_secret": self.consumer_secret
        }
        response = requests.post(token_url, data=payload)
        if response.status_code == 200:
            return response.json()["access_token"]
        return None
    
    def get_events(self, keyword=None, page=1, limit=6):
        """Récupère les événements avec pagination fonctionnelle"""
        if not self.access_token:
            return []
        
        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Accept": "application/json"
        }
        
        if keyword:
            url = f"{self.base_url}/events/search"
            params = {"keyword": keyword, "page": page}
        else:
            url = f"{self.base_url}/events"
            params = {"page": page}
        
        print(f"DEBUG - Recherche: {keyword}, Page: {page}")
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            # Extraction robuste des événements
            events = []
            if "response" in data:
                response_data = data["response"]
                if "results" in response_data:
                    results = response_data["results"]
                    if "event" in results:
                        events = results["event"]
                    elif "events" in results:
                        events = results["events"]
                elif "event" in response_data:
                    events = response_data["event"]
            elif "events" in data:
                events = data["events"]
            
            print(f"DEBUG - {len(events)} événements trouvés")
            return events[:limit]
        
        print(f"DEBUG - Erreur API: {response.status_code}")
        return []
    
    def format_event(self, event):
        """Extraction CORRECTE des données"""
        # DEBUG: Voir la structure
        # print(f"DEBUG Event keys: {list(event.keys())}")
        
        # Titre - méthode robuste
        title = "Activité"
        
        # Essayer translations d'abord
        translations = event.get("translations", {})
        if translations:
            for lang in ["fr", "nl", "en", "de"]:
                if lang in translations:
                    lang_data = translations[lang]
                    if isinstance(lang_data, dict):
                        if lang_data.get("title"):
                            title = lang_data["title"]
                            break
                        elif lang_data.get("name"):
                            title = lang_data["name"]
                            break
        
        # Si pas trouvé, essayer les champs directs
        if title == "Activité":
            for field in ["title", "name", "label"]:
                if event.get(field):
                    title = event[field]
                    break
        
        # Description
        description = ""
        for lang in ["fr", "nl", "en", "de"]:
            if lang in translations and isinstance(translations[lang], dict):
                desc = translations[lang].get("shortdescr") or translations[lang].get("description") or translations[lang].get("longdescr")
                if desc:
                    description = desc
                    break
        
        # Date
        dates = event.get("dates", [])
        start_date = "Date à confirmer"
        if dates and isinstance(dates, list) and len(dates) > 0:
            first_date = dates[0]
            if isinstance(first_date, dict):
                start_date = first_date.get("start", start_date)
                # Nettoyer le format de date
                if "T" in str(start_date):
                    start_date = str(start_date).split("T")[0]
        
        # Lieu
        place = event.get("place", {})
        place_name = "Bruxelles"
        if isinstance(place, dict):
            place_name = place.get("name", "Bruxelles")
            # Essayer les traductions du lieu
            if place_name == "Bruxelles" and "translations" in place:
                for lang in ["fr", "nl", "en"]:
                    if lang in place["translations"]:
                        place_trans = place["translations"][lang]
                        if isinstance(place_trans, dict) and place_trans.get("name"):
                            place_name = place_trans["name"]
                            break
        
        return {
            "title": title if title != "Activité" else "Activité à Bruxelles",
            "description": description[:100] + "..." if len(description) > 100 else description,
            "start_date": start_date,
            "location": place_name,
            "is_free": event.get("is_free", False)
        }

# ========== AGENT AVEC PAGINATION FONCTIONNELLE ==========
class WorkingSocialAgent:
    def __init__(self):
        self.api = BrusselsAPICorrected()
        self.current_state = {
            "keyword": None,
            "current_page": 1,
            "last_search": None
        }
    
    def chat(self, message: str) -> str:
        try:
            user_msg = message.lower().strip()
            
            # Réinitialiser si nouvelle recherche
            if user_msg not in ['autre', 'autres', 'suivant', 'encore', 'plus']:
                self.current_state["current_page"] = 1
                self.current_state["last_search"] = user_msg
            
            # Gestion "autres activités"
            if user_msg in ['autre', 'autres', 'suivant', 'encore', 'plus']:
                return self._handle_more_activities()
            
            # Détection du type de recherche
            keyword, intro = self._detect_search_type(user_msg)
            self.current_state["keyword"] = keyword
            
            # Recherche
            events = self.api.get_events(keyword, page=1, limit=6)
            
            if not events:
                return self._handle_no_results(keyword)
            
            return self._format_response(events, intro, is_first_page=True)
            
        except Exception as e:
            return f"❌ Erreur: {str(e)}"
    
    def _detect_search_type(self, message):
        """Détecte le type de recherche"""
        # Recherches spécifiques
        if any(word in message for word in ['film', 'cinéma', 'movie']):
            return "film", "🎬 **Films et cinéma à Bruxelles :**\n\n"
        elif any(word in message for word in ['musique', 'concert', 'festival', 'groupe']):
            return "concert", "🎵 **Concerts et musique :**\n\n"
        elif any(word in message for word in ['art', 'exposition', 'musée', 'galerie']):
            return "art", "🎨 **Art et expositions :**\n\n"
        elif any(word in message for word in ['sport', 'yoga', 'danse', 'fitness']):
            return "sport", "⚽ **Sports et activités :**\n\n"
        elif any(word in message for word in ['théâtre', 'spectacle', 'pièce']):
            return "théâtre", "🎭 **Théâtre et spectacles :**\n\n"
        elif any(word in message for word in ['cuisine', 'food', 'culinaire']):
            return "cuisine", "🍳 **Cuisine et gastronomie :**\n\n"
        elif any(word in message for word in ['gratuit', 'free']):
            return "gratuit", "🆓 **Activités gratuites :**\n\n"
        elif any(word in message for word in ['ennui', 'seul', 'solitude', 'social']):
            return None, "😊 **Activités pour socialiser :**\n\n"
        else:
            return None, "🎯 **Découvrez Bruxelles :**\n\n"
    
    def _handle_more_activities(self):
        """Gère la demande d'autres activités"""
        self.current_state["current_page"] += 1
        page = self.current_state["current_page"]
        keyword = self.current_state["keyword"]
        
        print(f"DEBUG - Page suivante: {page}, Keyword: {keyword}")
        
        events = self.api.get_events(keyword, page=page, limit=6)
        
        if not events:
            self.current_state["current_page"] = 1  # Reset
            return "📭 **Plus d'activités disponibles.**\n\n🎯 Essayez une nouvelle recherche !"
        
        if keyword:
            intro = f"🎯 **Plus d'activités {keyword} (page {page}) :**\n\n"
        else:
            intro = f"🎯 **Autres activités (page {page}) :**\n\n"
        
        return self._format_response(events, intro, is_first_page=False)
    
    def _handle_no_results(self, keyword):
        """Gère l'absence de résultats"""
        # Fallback: événements généraux
        events = self.api.get_events(None, page=1, limit=6)
        if events:
            if keyword:
                return f"🔍 **Aucun résultat pour '{keyword}', voici d'autres suggestions :**\n\n" + self._format_response(events, "", False)
            else:
                return "❌ **Aucun événement trouvé.** Voici des alternatives :\n\n" + self._format_response(events, "", False)
        else:
            return "❌ **Aucun événement disponible.**\n\n💡 Consultez https://agenda.brussels"
    
    def _format_response(self, events, intro, is_first_page=True):
        """Formate la réponse"""
        result = intro
        
        for i, event in enumerate(events, 1):
            formatted = self.api.format_event(event)
            result += f"**{i}. {formatted['title']}**\n"
            result += f"   📍 {formatted['location']}\n"
            result += f"   📅 {formatted['start_date']}\n"
            result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
            if formatted['description']:
                result += f"   📝 {formatted['description']}\n"
            result += "\n"
        
        # Message de fin contextuel
        if is_first_page:
            result += "💬 **Idéal pour rencontrer du monde !**\n"
            result += "🔄 **Dites 'autre' pour plus d'options**\n"
        else:
            result += "💬 **Encore plus de découvertes !**\n"
            result += "🔄 **Dites 'autre' pour continuer**\n"
        
        result += "🎯 **Nouvelle recherche :** Dites ce qui vous plaît"
        
        return result

# Dans create_chat_interface(), remplacez par :
# agent = WorkingSocialAgent()

In [32]:
"""Crée l'interface de chat corrigée"""
def create_chat_interface():

    # Initialisation de l'agent
    # agent = GuidedSocialAgent()
    # agent = SimpleSocialAgent()
    # agent = ImprovedSocialAgent()
    # agent = SmartSocialAgent()
    agent = WorkingSocialAgent()


    # Création des widgets
    chat_output = widgets.Output(layout={
        'border': '1px solid black', 
        'height': '400px', 
        'overflow_y': 'auto',
        'padding': '10px'
    })

    message_input = widgets.Text(
        placeholder='Tapez votre message ici...', 
        layout={'width': '70%'}
    )

    send_button = widgets.Button(
        description='Envoyer ✨', 
        button_style='success', 
        layout={'width': '20%'}
    )

    clear_button = widgets.Button(
        description='Effacer', 
        button_style='warning', 
        layout={'width': '10%'}
    )

    # Style CSS
    css_style = """
    <style>
    .chat-message { 
        margin: 10px 0; 
        padding: 12px; 
        border-radius: 15px; 
        max-width: 80%; 
        word-wrap: break-word;
    }
    .user-message { 
        background: #e3f2fd; 
        margin-left: 20%; 
        border-bottom-right-radius: 5px;
    }
    .bot-message { 
        background: #f5f5f5; 
        margin-right: 20%; 
        border-bottom-left-radius: 5px;
    }
    .message-sender { 
        font-weight: bold; 
        margin-bottom: 5px; 
        color: #333;
    }
    </style>
    """

    # Fonction pour envoyer un message
    def send_message(_):
        message = message_input.value.strip()
        if not message:
            return
        
        # Affiche le message utilisateur
        with chat_output:
            display(HTML(f"""
            <div class="chat-message user-message">
                <div class="message-sender">👤 Vous</div>
                {message}
            </div>
            """))
        
        # Désactive le bouton pendant le traitement
        send_button.disabled = True
        send_button.description = "Réflexion..."
        
        try:
            # Gère les commandes spéciales
            if message.lower() in ['recommencer', 'reset', 'nouvelle recherche', 'start']:
                agent.reset_conversation()
                response = "Conversation réinitialisée ! Prêt pour une nouvelle recherche 🔍"
            else:
                # Réponse normale de l'agent
                response = agent.chat(message)
            
            # Affiche la réponse
            with chat_output:
                display(HTML(f"""
                <div class="chat-message bot-message">
                    <div class="message-sender">🤖 Agent Social</div>
                    {response.replace(chr(10), '<br>')}
                </div>
                """))
                
        except Exception as e:
            with chat_output:
                display(HTML(f"""
                <div class="chat-message bot-message">
                    <div class="message-sender">❌ Erreur</div>
                    Désolé, une erreur s'est produite. Réessayez !
                </div>
                """))
            print(f"DEBUG Erreur: {e}")
        
        # Réactive le bouton
        send_button.disabled = False
        send_button.description = "Envoyer ✨"
        message_input.value = ''

    # Fonction pour effacer le chat
    def clear_chat(_):
        chat_output.clear_output()
        agent.reset_conversation()
        with chat_output:
            display(HTML("""
            <div class="chat-message bot-message">
                <div class="message-sender">🤖 Agent Social</div>
                Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. 
                Dites-moi ce qui vous intéresse ou comment vous vous sentez aujourd'hui.
            </div>
            """))

    # Fonction pour la touche Entrée
    def on_enter(_):
        send_message(None)

    # Assignation des événements
    send_button.on_click(send_message)
    clear_button.on_click(clear_chat)
    message_input.on_submit(on_enter)

    # Affichage de l'interface
    display(HTML(css_style))
    display(HTML("<h2>🤖 Social Connection Agent</h2>"))
    display(HTML("<p><em>Je vous aide à trouver des activités sociales à Bruxelles 🌟</em></p>"))
    display(chat_output)
    display(widgets.HBox([message_input, send_button, clear_button]))
    
    # Message de bienvenue initial
    with chat_output:
        display(HTML("""
        <div class="chat-message bot-message">
            <div class="message-sender">🤖 Agent Social</div>
            Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. <br><br>
            💡 <strong>Exemples de questions:</strong><br>
            • "Je me sens seul, que faire ?"<br>
            • "Je cherche des activités gratuites"<br>
            • "J'aime la musique, que me conseilles-tu ?"<br>
            • "Quels événements y a-t-il ce weekend ?"<br><br>
            Tapez votre message ci-dessous pour commencer ! 🚀
        </div>
        """))

# Lancement de l'interface
print("✅ Interface sociale prête !")
create_chat_interface()


✅ Interface sociale prête !


C:\Users\ziani\AppData\Local\Temp\ipykernel_21636\983104413.py:137: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  message_input.on_submit(on_enter)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

AttributeError: 'WorkingSocialAgent' object has no attribute 'reset_conversation'

#### OK WorkingSocialAgent est encore mieux que le précédent mais le filtrage ne fonctionne pas. Il n'arrive pas à catégoriser 
-------
#####

In [33]:
# ========== AGENT AVEC FILTRAGE INTELLIGENT ==========
class SmartFilterAgent:
    def __init__(self):
        self.api = BrusselsAPICorrected()
        self.current_state = {
            "filter_type": None,
            "current_page": 1,
            "all_events": []  # Cache des événements
        }
    
    def chat(self, message: str) -> str:
        try:
            user_msg = message.lower().strip()
            
            # Réinitialiser si nouvelle recherche
            if user_msg not in ['autre', 'autres', 'suivant', 'encore', 'plus']:
                self.current_state["current_page"] = 1
                self.current_state["all_events"] = []  # Vider le cache
            
            # Gestion "autres activités"
            if user_msg in ['autre', 'autres', 'suivant', 'encore', 'plus']:
                return self._handle_more_activities()
            
            # Détection du type de filtre
            filter_type, intro = self._detect_filter_type(user_msg)
            self.current_state["filter_type"] = filter_type
            
            # Charger les événements si nécessaire
            if not self.current_state["all_events"]:
                self.current_state["all_events"] = self._load_all_events()
            
            # Filtrer les événements
            filtered_events = self._filter_events(self.current_state["all_events"], filter_type)
            
            if not filtered_events:
                return self._handle_no_results(filter_type)
            
            # Paginer les résultats filtrés
            page_events = self._get_page_events(filtered_events, self.current_state["current_page"])
            
            return self._format_response(page_events, intro, is_first_page=True, total_pages=len(filtered_events)//6+1)
            
        except Exception as e:
            return f"❌ Erreur: {str(e)}"
    
    def _load_all_events(self, max_pages=5):
        """Charge plusieurs pages d'événements pour avoir plus de choix"""
        all_events = []
        for page in range(1, max_pages + 1):
            events = self.api.get_events(None, page=page, limit=25)  # Plus d'événements par page
            if events:
                all_events.extend(events)
            else:
                break
        print(f"DEBUG - {len(all_events)} événements chargés")
        return all_events
    
    def _detect_filter_type(self, message):
        """Détecte le type de filtre avec plus de mots-clés"""
        filter_map = {
            'film': {
                'keywords': ['film', 'cinéma', 'movie', 'projection', 'cinema', 'films', 'séance'],
                'intro': "🎬 **Films et cinéma à Bruxelles :**\n\n",
                'search_terms': ['cinema', 'film', 'movie']
            },
            'musique': {
                'keywords': ['musique', 'concert', 'festival', 'groupe', 'dj', 'chanson', 'rock', 'jazz', 'live', 'musical'],
                'intro': "🎵 **Concerts et musique :**\n\n",
                'search_terms': ['concert', 'musique', 'music', 'festival']
            },
            'art': {
                'keywords': ['art', 'exposition', 'musée', 'galerie', 'peinture', 'sculpture', 'dessin', 'artistique', 'artiste'],
                'intro': "🎨 **Art et expositions :**\n\n",
                'search_terms': ['art', 'exposition', 'museum', 'galerie']
            },
            'sport': {
                'keywords': ['sport', 'yoga', 'danse', 'fitness', 'randonnée', 'course', 'vélo', 'athlétisme', 'gym'],
                'intro': "⚽ **Sports et activités :**\n\n",
                'search_terms': ['sport', 'yoga', 'fitness', 'danse']
            },
            'theatre': {
                'keywords': ['théâtre', 'spectacle', 'pièce', 'comédie', 'drame', 'improvisation', 'scène'],
                'intro': "🎭 **Théâtre et spectacles :**\n\n",
                'search_terms': ['théâtre', 'theatre', 'spectacle']
            },
            'cuisine': {
                'keywords': ['cuisine', 'food', 'culinaire', 'atelier cuisine', 'chocolat', 'gastronomie', 'cooking', 'recette'],
                'intro': "🍳 **Cuisine et gastronomie :**\n\n",
                'search_terms': ['cuisine', 'cooking', 'chocolat', 'food']
            },
            'gratuit': {
                'keywords': ['gratuit', 'free', 'pas cher', 'gratuite'],
                'intro': "🆓 **Activités gratuites :**\n\n",
                'search_terms': []
            }
        }
        
        for filter_type, data in filter_map.items():
            if any(keyword in message for keyword in data['keywords']):
                return filter_type, data['intro']
        
        # Par défaut pour ennui/social
        if any(word in message for word in ['ennui', 'seul', 'solitude', 'social', 'rencontrer']):
            return 'social', "😊 **Activités pour socialiser :**\n\n"
        
        return None, "🎯 **Découvrez Bruxelles :**\n\n"
    
    def _filter_events(self, events, filter_type):
        """Filtre les événements localement"""
        if not filter_type or filter_type == 'social':
            return events[:30]  # Limiter pour social
            
        filtered = []
        
        for event in events:
            formatted = self.api.format_event(event)
            title = formatted['title'].lower()
            description = formatted['description'].lower()
            location = formatted['location'].lower()
            
            # Filtrage par type
            if filter_type == 'film':
                if any(term in title or term in description for term in 
                      ['film', 'cinéma', 'cinema', 'movie', 'projection', 'séance']):
                    filtered.append(event)
                    
            elif filter_type == 'musique':
                if any(term in title or term in description for term in 
                      ['concert', 'musique', 'music', 'festival', 'groupe', 'band', 'live', 'dj']):
                    filtered.append(event)
                    
            elif filter_type == 'art':
                if any(term in title or term in description or term in location for term in 
                      ['art', 'exposition', 'musée', 'museum', 'galerie', 'peinture', 'sculpture']):
                    filtered.append(event)
                    
            elif filter_type == 'sport':
                if any(term in title or term in description for term in 
                      ['sport', 'yoga', 'fitness', 'danse', 'course', 'vélo', 'gym', 'athlétisme']):
                    filtered.append(event)
                    
            elif filter_type == 'theatre':
                if any(term in title or term in description for term in 
                      ['théâtre', 'theatre', 'spectacle', 'pièce', 'comédie', 'improvisation']):
                    filtered.append(event)
                    
            elif filter_type == 'cuisine':
                if any(term in title or term in description for term in 
                      ['cuisine', 'cooking', 'chocolat', 'food', 'culinaire', 'atelier']):
                    filtered.append(event)
                    
            elif filter_type == 'gratuit':
                if formatted['is_free']:
                    filtered.append(event)
        
        print(f"DEBUG - Filtre '{filter_type}': {len(filtered)} résultats")
        return filtered
    
    def _get_page_events(self, events, page, per_page=6):
        """Paginer les événements filtrés"""
        start_idx = (page - 1) * per_page
        end_idx = start_idx + per_page
        return events[start_idx:end_idx]
    
    def _handle_more_activities(self):
        """Gère la demande d'autres activités"""
        if not self.current_state["all_events"]:
            self.current_state["all_events"] = self._load_all_events()
        
        self.current_state["current_page"] += 1
        page = self.current_state["current_page"]
        filter_type = self.current_state["filter_type"]
        
        # Re-filtrer si nécessaire
        filtered_events = self._filter_events(self.current_state["all_events"], filter_type)
        
        page_events = self._get_page_events(filtered_events, page)
        
        if not page_events:
            self.current_state["current_page"] = 1
            return "📭 **Plus d'activités de ce type.**\n\n🎯 Essayez une autre catégorie !"
        
        if filter_type:
            intro = f"🎯 **Plus d'activités {filter_type} (page {page}) :**\n\n"
        else:
            intro = f"🎯 **Autres activités (page {page}) :**\n\n"
        
        total_pages = len(filtered_events) // 6 + 1
        return self._format_response(page_events, intro, is_first_page=False, total_pages=total_pages)
    
    def _handle_no_results(self, filter_type):
        """Gère l'absence de résultats"""
        # Fallback: événements généraux
        if not self.current_state["all_events"]:
            self.current_state["all_events"] = self._load_all_events()
        
        events = self.current_state["all_events"][:6]
        
        if filter_type:
            return f"🔍 **Aucune activité '{filter_type}' trouvée, voici d'autres suggestions :**\n\n" + self._format_response(events, "", False, 1)
        else:
            return "❌ **Aucun événement trouvé.** Voici des alternatives :\n\n" + self._format_response(events, "", False, 1)
    
    def _format_response(self, events, intro, is_first_page=True, total_pages=1):
        """Formate la réponse"""
        result = intro
        
        for i, event in enumerate(events, 1):
            formatted = self.api.format_event(event)
            result += f"**{i}. {formatted['title']}**\n"
            result += f"   📍 {formatted['location']}\n"
            result += f"   📅 {formatted['start_date']}\n"
            result += f"   💰 {'🆓 Gratuit' if formatted['is_free'] else '💶 Payant'}\n"
            if formatted['description']:
                result += f"   📝 {formatted['description']}\n"
            result += "\n"
        
        # Message de fin contextuel
        if is_first_page:
            result += f"💬 **{len(events)} activités trouvées** • Page 1/{total_pages}\n"
            result += "🔄 **Dites 'autre' pour plus d'options**\n"
        else:
            result += f"💬 **Page {self.current_state['current_page']}/{total_pages}**\n"
            result += "🔄 **Dites 'autre' pour continuer**\n"
        
        result += "🎯 **Nouvelle recherche :** Dites ce qui vous plaît"
        
        return result

# Dans create_chat_interface(), remplacez par :
# agent = SmartFilterAgent()

In [34]:
"""Crée l'interface de chat corrigée"""
def create_chat_interface():

    # Initialisation de l'agent
    # agent = GuidedSocialAgent()
    # agent = SimpleSocialAgent()
    # agent = ImprovedSocialAgent()
    # agent = SmartSocialAgent()
    # agent = WorkingSocialAgent()
    agent = SmartFilterAgent()


    # Création des widgets
    chat_output = widgets.Output(layout={
        'border': '1px solid black', 
        'height': '400px', 
        'overflow_y': 'auto',
        'padding': '10px'
    })

    message_input = widgets.Text(
        placeholder='Tapez votre message ici...', 
        layout={'width': '70%'}
    )

    send_button = widgets.Button(
        description='Envoyer ✨', 
        button_style='success', 
        layout={'width': '20%'}
    )

    clear_button = widgets.Button(
        description='Effacer', 
        button_style='warning', 
        layout={'width': '10%'}
    )

    # Style CSS
    css_style = """
    <style>
    .chat-message { 
        margin: 10px 0; 
        padding: 12px; 
        border-radius: 15px; 
        max-width: 80%; 
        word-wrap: break-word;
    }
    .user-message { 
        background: #e3f2fd; 
        margin-left: 20%; 
        border-bottom-right-radius: 5px;
    }
    .bot-message { 
        background: #f5f5f5; 
        margin-right: 20%; 
        border-bottom-left-radius: 5px;
    }
    .message-sender { 
        font-weight: bold; 
        margin-bottom: 5px; 
        color: #333;
    }
    </style>
    """

    # Fonction pour envoyer un message
    def send_message(_):
        message = message_input.value.strip()
        if not message:
            return
        
        # Affiche le message utilisateur
        with chat_output:
            display(HTML(f"""
            <div class="chat-message user-message">
                <div class="message-sender">👤 Vous</div>
                {message}
            </div>
            """))
        
        # Désactive le bouton pendant le traitement
        send_button.disabled = True
        send_button.description = "Réflexion..."
        
        try:
            # Gère les commandes spéciales
            if message.lower() in ['recommencer', 'reset', 'nouvelle recherche', 'start']:
                agent.reset_conversation()
                response = "Conversation réinitialisée ! Prêt pour une nouvelle recherche 🔍"
            else:
                # Réponse normale de l'agent
                response = agent.chat(message)
            
            # Affiche la réponse
            with chat_output:
                display(HTML(f"""
                <div class="chat-message bot-message">
                    <div class="message-sender">🤖 Agent Social</div>
                    {response.replace(chr(10), '<br>')}
                </div>
                """))
                
        except Exception as e:
            with chat_output:
                display(HTML(f"""
                <div class="chat-message bot-message">
                    <div class="message-sender">❌ Erreur</div>
                    Désolé, une erreur s'est produite. Réessayez !
                </div>
                """))
            print(f"DEBUG Erreur: {e}")
        
        # Réactive le bouton
        send_button.disabled = False
        send_button.description = "Envoyer ✨"
        message_input.value = ''

    # Fonction pour effacer le chat
    def clear_chat(_):
        chat_output.clear_output()
        agent.reset_conversation()
        with chat_output:
            display(HTML("""
            <div class="chat-message bot-message">
                <div class="message-sender">🤖 Agent Social</div>
                Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. 
                Dites-moi ce qui vous intéresse ou comment vous vous sentez aujourd'hui.
            </div>
            """))

    # Fonction pour la touche Entrée
    def on_enter(_):
        send_message(None)

    # Assignation des événements
    send_button.on_click(send_message)
    clear_button.on_click(clear_chat)
    message_input.on_submit(on_enter)

    # Affichage de l'interface
    display(HTML(css_style))
    display(HTML("<h2>🤖 Social Connection Agent</h2>"))
    display(HTML("<p><em>Je vous aide à trouver des activités sociales à Bruxelles 🌟</em></p>"))
    display(chat_output)
    display(widgets.HBox([message_input, send_button, clear_button]))
    
    # Message de bienvenue initial
    with chat_output:
        display(HTML("""
        <div class="chat-message bot-message">
            <div class="message-sender">🤖 Agent Social</div>
            Bonjour ! Je suis là pour vous aider à trouver des activités sociales à Bruxelles. <br><br>
            💡 <strong>Exemples de questions:</strong><br>
            • "Je me sens seul, que faire ?"<br>
            • "Je cherche des activités gratuites"<br>
            • "J'aime la musique, que me conseilles-tu ?"<br>
            • "Quels événements y a-t-il ce weekend ?"<br><br>
            Tapez votre message ci-dessous pour commencer ! 🚀
        </div>
        """))

# Lancement de l'interface
print("✅ Interface sociale prête !")
create_chat_interface()


✅ Interface sociale prête !


C:\Users\ziani\AppData\Local\Temp\ipykernel_21636\330569301.py:138: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  message_input.on_submit(on_enter)


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…